<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Space X  Falcon 9 First Stage Landing Prediction**


## Web scraping Falcon 9 and Falcon Heavy Launches Records from Wikipedia


Estimated time needed: **40** minutes


In this lab, you will be performing web scraping to collect Falcon 9 historical launch records from a Wikipedia page titled `List of Falcon 9 and Falcon Heavy launches`

https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/Falcon9_rocket_family.svg)


Falcon 9 first stage will land successfully


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/landing_1.gif)


Several examples of an unsuccessful landing are shown here:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/crash.gif)


More specifically, the launch records are stored in a HTML table shown below:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/falcon9-launches-wiki.png)


  ## Objectives
Web scrap Falcon 9 launch records with `BeautifulSoup`: 
- Extract a Falcon 9 launch records HTML table from Wikipedia
- Parse the table and convert it into a Pandas data frame


First let's import required packages for this lab


In [1]:
!pip3 install beautifulsoup4
!pip3 install requests

In [2]:
import sys

import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

and we will provide some helper functions for you to process web scraped HTML table


In [3]:
def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name    


To keep the lab tasks consistent, you will be asked to scrape the data from a snapshot of the  `List of Falcon 9 and Falcon Heavy launches` Wikipage updated on
`9th June 2021`


In [4]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

Next, request the HTML page from the above URL and get a `response` object


### TASK 1: Request the Falcon9 Launch Wiki page from its URL


First, let's perform an HTTP GET method to request the Falcon9 Launch HTML page, as an HTTP response.


In [5]:
import requests

# URL of the Wikipedia page
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

# Send a GET request to the URL
response = requests.get(static_url)

# Check if the request was successful
if response.status_code == 200:
    print("Successfully retrieved the webpage!")
else:
    print(f"Failed to retrieve the webpage. Status code: {response.status_code}")

Successfully retrieved the webpage!


Create a `BeautifulSoup` object from the HTML `response`


In [6]:
from bs4 import BeautifulSoup

# Create a BeautifulSoup object from the HTML response content
soup = BeautifulSoup(response.content, 'html.parser')

# Print the title of the page to verify that the BeautifulSoup object was created correctly
print(soup.title.string)

List of Falcon 9 and Falcon Heavy launches - Wikipedia


Print the page title to verify if the `BeautifulSoup` object was created properly 


In [7]:
print(soup.title.string)

List of Falcon 9 and Falcon Heavy launches - Wikipedia


### TASK 2: Extract all column/variable names from the HTML table header


Next, we want to collect all relevant column names from the HTML table header


Let's try to find all tables on the wiki page first. If you need to refresh your memory about `BeautifulSoup`, please check the external reference link towards the end of this lab


In [8]:
# Use the find_all function in the BeautifulSoup object to find all tables
html_tables = soup.find_all('table')

# Print the number of tables found to verify
print(f"Number of tables found: {len(html_tables)}")

Number of tables found: 25


Starting from the third table is our target table contains the actual launch records.


In [9]:
# The third table (index 2) is our target table that contains the actual launch records
first_launch_table = html_tables[2]

# Print the content of the third table to verify
print(first_launch_table.prettify())

<table class="wikitable plainrowheaders collapsible" style="width: 100%;">
 <tbody>
  <tr>
   <th scope="col">
    Flight No.
   </th>
   <th scope="col">
    Date and
    <br/>
    time (
    <a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">
     UTC
    </a>
    )
   </th>
   <th scope="col">
    <a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">
     Version,
     <br/>
     Booster
    </a>
    <sup class="reference" id="cite_ref-booster_11-0">
     <a href="#cite_note-booster-11">
      <span class="cite-bracket">
       [
      </span>
      b
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
   </th>
   <th scope="col">
    Launch site
   </th>
   <th scope="col">
    Payload
    <sup class="reference" id="cite_ref-Dragon_12-0">
     <a href="#cite_note-Dragon-12">
      <span class="cite-bracket">
       [
      </span>
      c
      <span class="cite-bracket">
       ]
  

You should able to see the columns names embedded in the table header elements `<th>` as follows:


```
<tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12">[c]</a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>
```


Next, we just need to iterate through the `<th>` elements and apply the provided `extract_column_from_header()` to extract column name one by one


In [10]:
# Define the list to hold the column names
column_names = []

# Iterate through each <th> element in the table header and extract the column name
for th in first_launch_table.find_all('th'):
    name = extract_column_from_header(th)
    if name is not None and len(name) > 0:
        column_names.append(name)

# Print the extracted column names
print("Extracted Column Names:")
print(column_names)

Extracted Column Names:
['Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome']


Check the extracted column names


In [11]:
# Print the extracted column names to verify
print("Extracted Column Names:")
print(column_names)
#Extracted Column Names:
['Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome']

Extracted Column Names:
['Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome']


['Flight No.',
 'Date and time ( )',
 'Launch site',
 'Payload',
 'Payload mass',
 'Orbit',
 'Customer',
 'Launch outcome']

## TASK 3: Create a data frame by parsing the launch HTML tables


We will create an empty dictionary with keys from the extracted column names in the previous task. Later, this dictionary will be converted into a Pandas dataframe


In [12]:
# Create a dictionary with keys from the extracted column names
launch_dict = dict.fromkeys(column_names)

# Remove the irrelevant column "Date and time ( )"
del launch_dict['Date and time ( )']

# Initialize each key in the dictionary with an empty list
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []

# Add additional columns that were not in the original table headers
launch_dict['Version Booster'] = []
launch_dict['Booster landing'] = []
launch_dict['Date'] = []
launch_dict['Time'] = []

# Print the initialized dictionary keys to verify
print("Initialized launch_dict keys:")
print(launch_dict.keys())

Initialized launch_dict keys:
dict_keys(['Flight No.', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome', 'Version Booster', 'Booster landing', 'Date', 'Time'])


Next, we just need to fill up the `launch_dict` with launch records extracted from table rows.


Usually, HTML tables in Wiki pages are likely to contain unexpected annotations and other types of noises, such as reference links `B0004.1[8]`, missing values `N/A [e]`, inconsistent formatting, etc.


To simplify the parsing process, we have provided an incomplete code snippet below to help you to fill up the `launch_dict`. Please complete the following code snippet with TODOs or you can choose to write your own logic to parse all launch tables:


In [13]:
extracted_row = 0

# Extract each table 
for table_number, table in enumerate(soup.find_all('table', "wikitable plainrowheaders collapsible")):
    # Get each table row 
    for rows in table.find_all("tr"):
        # Check to see if the first table heading is a number corresponding to a launch number 
        if rows.th:
            if rows.th.string:
                flight_number = rows.th.string.strip()
                flag = flight_number.isdigit()
        else:
            flag = False
        
        # Get table elements 
        row = rows.find_all('td')
        
        # If it is a number, save cells in the dictionary 
        if flag:
            extracted_row += 1
            
            # Flight Number value
            launch_dict['Flight No.'].append(flight_number)
            
            # Date and Time values
            datatimelist = date_time(row[0])
            launch_dict['Date'].append(datatimelist[0].strip(','))
            launch_dict['Time'].append(datatimelist[1])
              
            # Booster version
            bv = booster_version(row[1])
            if not(bv):
                bv = row[1].a.string if row[1].a else "N/A"
            launch_dict['Version Booster'].append(bv)
            
            # Launch Site
            launch_site = row[2].a.string if row[2].a else "N/A"
            launch_dict['Launch site'].append(launch_site)
            
            # Payload
            payload = row[3].a.string if row[3].a else "N/A"
            launch_dict['Payload'].append(payload)
            
            # Payload Mass
            payload_mass = get_mass(row[4])
            launch_dict['Payload mass'].append(payload_mass)
            
            # Orbit
            orbit = row[5].a.string if row[5].a else "N/A"
            launch_dict['Orbit'].append(orbit)
            
            # Customer
            customer = row[6].a.string if row[6].a else "N/A"
            launch_dict['Customer'].append(customer)
            
            # Launch outcome
            launch_outcome = list(row[7].strings)[0]
            launch_dict['Launch outcome'].append(launch_outcome)
            
            # Booster landing
            booster_landing = landing_status(row[8])
            launch_dict['Booster landing'].append(booster_landing)

# Check the number of rows extracted
print(f"Number of rows extracted: {extracted_row}")

Number of rows extracted: 121


After you have fill in the parsed launch record values into `launch_dict`, you can create a dataframe from it.


In [14]:
for key in launch_dict:
    print(f"{key}: {len(launch_dict[key])}")

Flight No.: 121
Launch site: 121
Payload: 121
Payload mass: 121
Orbit: 121
Customer: 121
Launch outcome: 121
Version Booster: 121
Booster landing: 121
Date: 121
Time: 121


We can now export it to a <b>CSV</b> for the next section, but to make the answers consistent and in case you have difficulties finishing this lab. 

Following labs will be using a provided dataset to make each lab independent. 


In [15]:
import requests
import pandas as pd

# Perform the GET request on the SpaceX API
response = requests.get("https://api.spacexdata.com/v4/launches")

# Convert the response to a DataFrame
data = pd.json_normalize(response.json())

# Extract the first year from the 'static_fire_date_utc' column
first_year = pd.to_datetime(data['static_fire_date_utc']).dt.year.iloc[0]
print(first_year)

2006.0


<code>df.to_csv('spacex_web_scraped.csv', index=False)</code>


In [16]:
# Filter out Falcon 1 launches
falcon9_launches = data[data['rocket'] == 'Falcon 9']

# Count the number of Falcon 9 launches
falcon9_count = len(falcon9_launches)
print(falcon9_count)

0


In [30]:
# Check the type of the 'cores' column
print(type(data['cores']))

# Print out a sample of the 'cores' column to understand its structure
print(data['cores'].head())

<class 'pandas.core.series.Series'>
0    [{'core': '5e9e289df35918033d3b2623', 'flight'...
1    [{'core': '5e9e289ef35918416a3b2624', 'flight'...
2    [{'core': '5e9e289ef3591814873b2625', 'flight'...
3    [{'core': '5e9e289ef3591855dc3b2626', 'flight'...
4    [{'core': '5e9e289ef359184f103b2627', 'flight'...
Name: cores, dtype: object


In [31]:
# Normalize the 'cores' column if it contains nested data
core_info_normalized = pd.json_normalize(data['cores'])

# Check the columns in the normalized DataFrame
print(core_info_normalized.columns)

RangeIndex(start=0, stop=3, step=1)


In [32]:
# If 'landing_pad' exists, count the missing values
if 'landing_pad' in core_info_normalized.columns:
    missing_landingPad = core_info_normalized['landing_pad'].isna().sum()
    print(f"Missing landing_pad values: {missing_landingPad}")
else:
    print("The 'landing_pad' column was not found.")

The 'landing_pad' column was not found.


In [18]:
# Print the column names to verify if 'landingPad' exists
print(data.columns)

Index(['static_fire_date_utc', 'static_fire_date_unix', 'net', 'window',
       'rocket', 'success', 'failures', 'details', 'crew', 'ships', 'capsules',
       'payloads', 'launchpad', 'flight_number', 'name', 'date_utc',
       'date_unix', 'date_local', 'date_precision', 'upcoming', 'cores',
       'auto_update', 'tbd', 'launch_library_id', 'id', 'fairings.reused',
       'fairings.recovery_attempt', 'fairings.recovered', 'fairings.ships',
       'links.patch.small', 'links.patch.large', 'links.reddit.campaign',
       'links.reddit.launch', 'links.reddit.media', 'links.reddit.recovery',
       'links.flickr.small', 'links.flickr.original', 'links.presskit',
       'links.webcast', 'links.youtube_id', 'links.article', 'links.wikipedia',
       'fairings'],
      dtype='object')


In [29]:
# Normalize the 'cores' column to extract landing pad information
core_info = pd.json_normalize(data['cores'])

# Extract the 'landing_pad' column
landing_pad_column = core_info['landing_pad']

# Check for missing values in the 'landing_pad' column
missing_landingPad = landing_pad_column.isna().sum()
print(missing_landingPad)

KeyError: 'landing_pad'

In [20]:
from bs4 import BeautifulSoup

# Assuming 'soup' is already created from the Falcon 9 Wikipedia page
print(soup.title)

<title>List of Falcon 9 and Falcon Heavy launches - Wikipedia</title>


In [26]:
# Assuming the column containing the rocket name is named 'rocket' or something similar

# First, let's check if there's a column related to rocket names
print(data.columns)

# Filter out Falcon 1 launches
falcon9_launches = data[data['rocket'] == 'Falcon 9']

# Count the number of Falcon 9 launches
falcon9_count = len(falcon9_launches)
print(falcon9_count)

Index(['static_fire_date_utc', 'static_fire_date_unix', 'net', 'window',
       'rocket', 'success', 'failures', 'details', 'crew', 'ships', 'capsules',
       'payloads', 'launchpad', 'flight_number', 'name', 'date_utc',
       'date_unix', 'date_local', 'date_precision', 'upcoming', 'cores',
       'auto_update', 'tbd', 'launch_library_id', 'id', 'fairings.reused',
       'fairings.recovery_attempt', 'fairings.recovered', 'fairings.ships',
       'links.patch.small', 'links.patch.large', 'links.reddit.campaign',
       'links.reddit.launch', 'links.reddit.media', 'links.reddit.recovery',
       'links.flickr.small', 'links.flickr.original', 'links.presskit',
       'links.webcast', 'links.youtube_id', 'links.article', 'links.wikipedia',
       'fairings'],
      dtype='object')
0


In [21]:
# Check the unique values in the 'rocket' column
print(data['rocket'].unique())

['5e9d0d95eda69955f709d1eb' '5e9d0d95eda69973a809d1ec'
 '5e9d0d95eda69974db09d1ed']


In [22]:
# Replace 'CORRECT_VALUE' with the identified value
falcon9_launches = data[data['rocket'] == 'CORRECT_VALUE']

# Count the number of Falcon 9 launches
falcon9_count = len(falcon9_launches)
print(falcon9_count)

0


In [23]:
# Print all column names to verify the presence of 'landingPad'
print(data.columns)

# If 'landingPad' exists, calculate the number of missing values
if 'landingPad' in data.columns:
    missing_landingPad = data['landingPad'].isna().sum()
    print(missing_landingPad)
else:
    print("The 'landingPad' column does not exist in the DataFrame.")

Index(['static_fire_date_utc', 'static_fire_date_unix', 'net', 'window',
       'rocket', 'success', 'failures', 'details', 'crew', 'ships', 'capsules',
       'payloads', 'launchpad', 'flight_number', 'name', 'date_utc',
       'date_unix', 'date_local', 'date_precision', 'upcoming', 'cores',
       'auto_update', 'tbd', 'launch_library_id', 'id', 'fairings.reused',
       'fairings.recovery_attempt', 'fairings.recovered', 'fairings.ships',
       'links.patch.small', 'links.patch.large', 'links.reddit.campaign',
       'links.reddit.launch', 'links.reddit.media', 'links.reddit.recovery',
       'links.flickr.small', 'links.flickr.original', 'links.presskit',
       'links.webcast', 'links.youtube_id', 'links.article', 'links.wikipedia',
       'fairings'],
      dtype='object')
The 'landingPad' column does not exist in the DataFrame.


In [24]:
# Count the number of missing values in the 'launchpad' column
missing_launchpad = data['launchpad'].isna().sum()
print(f"Missing values in 'launchpad': {missing_launchpad}")

Missing values in 'launchpad': 0


<a href="https://www.linkedin.com/in/yan-luo-96288783/">Yan Luo</a>


In [25]:
# Count the number of missing values in the 'launchpad' column using .isnull()
missing_launchpad = data['launchpad'].isnull().sum()
print(f"Missing values in 'launchpad': {missing_launchpad}")

Missing values in 'launchpad': 0


<a href="https://www.linkedin.com/in/nayefaboutayoun/">Nayef Abou Tayoun</a>


In [26]:
# Assuming the column name is 'Booster Version'
falcon9_data = data[data['Booster Version'] == 'Falcon 9']

# Count the number of Falcon 9 launches
falcon9_launch_count = falcon9_data.shape[0]
print(falcon9_launch_count)

KeyError: 'Booster Version'

<!--
## Change Log
-->


In [27]:
# Replace 'landingPad' with the correct column name if necessary
missing_landingPad_count = data['landingPad'].isna().sum()
print(missing_landingPad_count)

KeyError: 'landingPad'

<!--
| Date (YYYY-MM-DD) | Version | Changed By | Change Description      |
| ----------------- | ------- | ---------- | ----------------------- |
| 2021-06-09        | 1.0     | Yan Luo    | Tasks updates           |
| 2020-11-10        | 1.0     | Nayef      | Created the initial version |
-->


Copyright © 2021 IBM Corporation. All rights reserved.
